## 배차 진짜 시작!!!!

https://developers.google.com/optimization/assignment/assignment_example 와 재연학생이 짠 코드를 참고하여 만들었습니다.  

1. loop문을 사용한 곳의 속도개선이 필요해 보임
2. Dispatch 알고리즘이 빠르게 동작할 수 있도록 택시와 승객의 데이터 프레임 수정 필요


### 1. 패키지 불러오기

In [9]:
import pandas as pd
import folium
import osmnx as ox
from numpy import random
from shapely.geometry import Point
import warnings 
import polyline
import folium
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from ortools.linear_solver import pywraplp

warnings.filterwarnings("ignore")

In [42]:
taxi_locations = pd.read_pickle('data/taxi_locations.pkl')
passenger_locations = pd.read_pickle('data/passenger_locations.pkl')

### 2. 데이터 전처리

Shapely 포맷이 아닌 숫자형태로 X,Y 좌표를 가지고 있는게 좋을 것 같음.  
랜덤 수요 생성할 때 이렇게 생성하는 코드로 수정할 필요가 있음.  
추후에 시각화를 할 때 Shapely Point 형식으로 변환하는게 어떨지

In [43]:
taxi_locations['x'] = [point.x for point in taxi_locations.geometry.values]
taxi_locations['y'] = [point.y for point in taxi_locations.geometry.values]

In [44]:
passenger_locations['dprt_x'] = [point.x for point in passenger_locations.start_point.values]
passenger_locations['dprt_y'] = [point.y for point in passenger_locations.start_point.values]
passenger_locations['arrvl_x'] = [point.x for point in passenger_locations.end_point.values]
passenger_locations['arrvl_y'] = [point.y for point in passenger_locations.end_point.values]

승객 생성할 때 시간 값을 분단위로 바꾸어 줌.  
시간 *60 + 분으로 계산했으며 따라서 1일은 0분 ~ 1440분까지의 정수값으로 표현됨  
이렇게 사용할지, 아니면 datetime 변수로 사용할지는 향후 결정 **(의견주세요)**

In [56]:
passenger_test = passenger_locations[['no', 
                                      'receipttime_time',
                                      "dprt_x",
                                      'dprt_y',
                                      'arrvl_x',
                                      'arrvl_y']].copy()

passenger_test['receipttime_time'] = pd.to_datetime(passenger_test['receipttime_time'], format='%H:%M:%S')

passenger_test['time'] = passenger_test['receipttime_time'].dt.minute + passenger_test['receipttime_time'].dt.hour*60

# 오후 12시를 예시로 추출해 봄
# 배차는 매 분마다 수행이 되도록
# 매초단위로 수행할 경우 최적화를 돌리는게 큰 의미가 없음
passenger_test = passenger_test[passenger_test['time']==720]
passenger_test.reset_index(drop=True, inplace=True)

### 3. OSRM을 사용한 두 지점간 최단시간 계산

Travel time을 가장 최소화하도록 배차를 수행  
시각화를 하기 전엔 굳이 Route를 출력할 필요가 없기 때문에 travel time 값만 추출

In [57]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [58]:
# 출발지와 목적지 경도(x,lon), 위도(y,lat)가 주어졌을 때 통행시간을 뽑는 함수
# Dispatch Algorithm을 작동할 때 사용
# 경로를 알 필요가 없고 통행시간만 뽑을 때 사용

def get_travel_time(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://127.0.0.1:5000/route/v1/driving/"
    r = session.get(url + loc)
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    duration = res['routes'][0]['duration']

    return duration

### 4. 최적화를 통한 Dispatch 수행

최적화 알고리즘이 도는 것은 매우 빠르나, cost matrix(최소 통행 시간) 계산하는 것이 오래 걸림  
이곳 Loop 문을 더 빠르게 개선할 필요가 있음

In [59]:
num_taxis = len(taxi_locations)
num_passenger = len(passenger_test)

In [60]:
#Declare the MIP solver
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [85]:
#Calculate cost matrix
# 모든 빈택시와 모든 승객들에 대해서 각각의 통행시간을 계산
# 속도가 너무 느려서 추후 개선 필요

costs = []
for i in range(num_taxis):
    cost = []
    for j in range(num_passenger):
        cost.append(get_travel_time(taxi_locations['x'].iloc[i],
                                      taxi_locations['y'].iloc[i],
                                      passenger_test['dprt_x'].iloc[j],
                                      passenger_test['dprt_y'].iloc[j]))
    costs.append(cost)
    
# apply 사용해봤는데 더 느림..

# costs = []
# for i in range(num_taxis):
#         cost_by_taxi = passenger_test.apply(lambda passenger: get_travel_time(taxi_locations['x'].iloc[i],
#                                                                               taxi_locations['y'].iloc[i],
#                                                                               passenger['dprt_x'],
#                                                                               passenger['dprt_y']),axis=1)
#         costs.append(cost_by_taxi)

In [100]:
num_taxis = len(taxi_locations)
num_passenger = len(passenger_test)

In [101]:
#Declare the MIP solver
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [102]:
#Create the variables
# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_taxis):
    for j in range(num_passenger):
        x[i, j] = solver.IntVar(0, 1, '')

In [103]:
#Create the constraints
# Each worker is assigned to at most 1 task.
for i in range(num_taxis):
    solver.Add(solver.Sum([x[i, j] for j in range(num_passenger)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_passenger):
    solver.Add(solver.Sum([x[i, j] for i in range(num_taxis)]) == 1)

In [104]:
#Create the objective function
objective_terms = []
for i in range(num_taxis):
    for j in range(num_passenger):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

In [105]:
#Invoke the solver
status = solver.Solve()

In [110]:
# Print solution
taxi_iloc = []
passenger_iloc = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    #print('Total distance = ', solver.Objective().Value(), '\n')
    for i in range(num_taxis):
        for j in range(num_passenger):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print('taxi %d assigned to passenger %d.  duration(s) = %d' %
                    (i, j, costs[i][j]))  
                taxi_iloc.append(i) 
                passenger_iloc.append(j)

taxi 18 assigned to passenger 12.  duration(s) = 198
taxi 23 assigned to passenger 15.  duration(s) = 134
taxi 123 assigned to passenger 11.  duration(s) = 116
taxi 125 assigned to passenger 3.  duration(s) = 44
taxi 142 assigned to passenger 5.  duration(s) = 78
taxi 168 assigned to passenger 4.  duration(s) = 45
taxi 170 assigned to passenger 2.  duration(s) = 109
taxi 192 assigned to passenger 18.  duration(s) = 71
taxi 230 assigned to passenger 16.  duration(s) = 185
taxi 241 assigned to passenger 0.  duration(s) = 170
taxi 277 assigned to passenger 10.  duration(s) = 91
taxi 289 assigned to passenger 8.  duration(s) = 84
taxi 384 assigned to passenger 7.  duration(s) = 106
taxi 386 assigned to passenger 1.  duration(s) = 22
taxi 415 assigned to passenger 14.  duration(s) = 148
taxi 447 assigned to passenger 13.  duration(s) = 22
taxi 450 assigned to passenger 6.  duration(s) = 14
taxi 453 assigned to passenger 9.  duration(s) = 80
taxi 488 assigned to passenger 17.  duration(s) = 

In [111]:
taxi_locations.iloc[taxi_iloc]

,Taxi_ID,boarding_status,geometry,x,y
18,18,0,POINT (127.10695427377811 37.46294060095353),127.106954,37.462941
23,23,0,POINT (127.05883800000001 37.5095751),127.058838,37.509575
123,123,0,POINT (127.10033025000001 37.4907692),127.100330,37.490769
125,125,0,POINT (127.02143386666667 37.5819184),127.021434,37.581918
142,142,0,POINT (126.92379105 37.4804281),126.923791,37.480428
168,168,0,POINT (126.89741520832112 37.492342970393395),126.897415,37.492343
170,170,0,POINT (127.00994081033251 37.55269794811593),127.009941,37.552698
192,192,0,POINT (126.83102432520498 37.538665560452216),126.831024,37.538666
230,230,0,POINT (127.00349645689772 37.52710349337322),127.003496,37.527103
241,241,0,POINT (126.94649190000001 37.5565395),126.946492,37.556539


In [112]:
passenger_test.iloc[passenger_iloc]

,no,receipttime_time,dprt_x,dprt_y,arrvl_x,arrvl_y,time
12,7879,1900-01-01 12:00:00,127.097136,37.467907,127.039443,37.487051,720
15,7907,1900-01-01 12:00:09,127.061140,37.516790,127.090539,37.457928,720
11,8194,1900-01-01 12:00:00,127.106021,37.488592,127.128593,37.528166,720
3,3410,1900-01-01 12:00:00,127.024093,37.581608,127.018260,37.619924,720
5,8241,1900-01-01 12:00:00,126.917828,37.478020,126.973309,37.406337,720
4,7817,1900-01-01 12:00:00,126.900601,37.495217,126.904977,37.519747,720
2,5613,1900-01-01 12:00:00,127.015714,37.557265,127.004545,37.575856,720
18,4963,1900-01-01 12:00:57,126.826958,37.540548,126.809667,37.565338,720
16,8258,1900-01-01 12:00:34,127.012890,37.538966,126.985385,37.545793,720
0,8098,1900-01-01 12:00:00,126.947880,37.568177,126.920403,37.570384,720


In [ ]:
def dispatch_taxi_to_passenger(empty_vehicle,passenger_data,assigned_vehicle,time):
    num_taxis = len(empty_vehicle)
    num_passenger = len(passenger_data)
    
    
    return(updated_empty_vehicle, updated_passenger_data, updated_assigned_vehicle)
    